In [164]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [244]:
final_df_modified = pd.read_csv("./counties.finalfeatures.timeseries.csv")

In [245]:
# final_df_modified['date'] = pd.DatetimeIndex(final_df_modified['date'].values)
# final_df_modified['date']

In [246]:
final_df_modified

,date,fips,state,county,actuals.cases,actuals.newCases,actuals.deaths,actuals.newDeaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.testPositivityRatio
0,2020-04-09,2013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
1,2020-04-10,2013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
2,2020-04-11,2013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
3,2020-04-12,2013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
4,2020-04-13,2013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131347,2022-01-07,56045,WY,Weston County,1263.0,2.0,16.0,0.0,17.0,0.347,0.389,1.13,2,2,3,0.087
2131348,2022-01-08,56045,WY,Weston County,1263.0,0.0,16.0,0.0,18.9,0.349,0.390,1.15,2,2,3,NaN
2131349,2022-01-09,56045,WY,Weston County,1263.0,0.0,16.0,0.0,20.8,0.349,0.390,1.17,2,2,3,NaN
2131350,2022-01-10,56045,WY,Weston County,1268.0,5.0,16.0,0.0,22.7,0.349,0.390,1.19,2,2,3,NaN


In [168]:
final_df_grouped = final_df_modified.groupby(['county'])

In [253]:
# for index in range(final_df_modified.shape[1]):
    
#     print('Column Number : ', index)
     
#     # Select column by index position using iloc[]
#     columnSeriesObj = final_df_modified.iloc[:, index]
#     #print("columnSeriesObj", final_df_modified.iloc[:, 3])
#     #print('Column Contents : ', columnSeriesObj.values)
#     county_list = final_df_modified.iloc[:, 3]
#     print(county_list.tolist())
#     #for index in county_list:
#      #   county_list[index] == county_list[index+1]
        
        

In [415]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
global count_cleaned_df
global error_message
error_message= []
from tqdm import tqdm

def fill_missing_days_with_NA(df):
    df.index = pd.DatetimeIndex(df['date'].values)
    final_df = df.asfreq('D')
    final_df['date'] = final_df.index
    return final_df


def data_preprocessing(df): 
    columns_to_exclude = ['date', 'riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    new_df = df.loc[:, ~df.columns.isin(columns_to_exclude)]
    
    #Filling missing values using backfill and forwardfill
    new_df.fillna(method="backfill", axis=None, inplace=True)
    new_df.fillna(method="ffill", axis=None, inplace=True)
    
    #Normalization using MinMaxScaler
    cols_to_norm = ['riskLevels.overall', 'riskLevels.caseDensity', 'cdcTransmissionLevel']
    scaler = MinMaxScaler()
    new_df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])
    
    #Copying Date
    new_df['date'] = df.date

    return new_df

def population_feature_engineering(df,fip):
    # df['metrics.vaccinationsCompletedRatioDensity'] = fe_df["metrics.vaccinationsCompletedRatio"]/counties_pop_df.loc[fip]["population"]
    # df['metrics.vaccinationsInitiatedRatioDensity'] = fe_df["metrics.vaccinationsInitiatedRatio"]/counties_pop_df.loc[fip]["population"]
    df["metrics.deathDensity"] = df['actuals.deaths']/counties_pop_df.loc[fip]["population"]
    return df

def seven_day_rolling_average(df): #Rolling only for non-cumulative features

    # rolling_7d_df = pd.DataFrame()
    df['date'] = pd.DatetimeIndex(df['date'].values)
    # print(df)
    rolling_7d_df = df.groupby(pd.Grouper(key='date', axis=0, freq='7D', sort=True)).mean()
    # print(rolling_7d_df)
    return rolling_7d_df


def merge_cumulative_noncumulative(rolling_7d_df,non_rolling_input_df):
    merged_df = pd.merge(rolling_7d_df,non_rolling_input_df, left_index=True,right_index=True)
             
    return merged_df


def clean_fips(df):
    df['fips']=(df['fips']/7).astype(int)
    
# def check_inconsistencies(df):
#     fips_value = df['fips'].tolist()[0]
#     global error_message
    
#     subset_df = df[['actuals.newCases','actuals.cases']]
#     previous_cumulative_cases = 0
#     previous_new_cases = 0
#     for index, row in subset_df.iterrows():
#         if previous_cumulative_cases + previous_new_cases > row['actuals.cases']:
#             message = f"Issue at time point: {index} for fips={fips_value}"
#             error_message.append(message)
        
#         previous_cumulative_cases = row['actuals.cases']
#         previous_new_cases = row['actuals.newCases']

original_csv = pd.read_csv("./counties.finalfeatures.timeseries.csv")
original_csv.drop(['actuals.newCases','actuals.newDeaths'],axis=1,inplace=True)
fips_list = list(set(original_csv['fips'].tolist()))
new_df_list = []
i=0
for fip in tqdm(fips_list):
    county_df = original_csv[original_csv['fips'] == fip]
    
    #STEP 1
    county_df = fill_missing_days_with_NA(county_df) 
    #dataframe is set to DateTimeIndex
    
    #STEP 2
    new_county_df = data_preprocessing(county_df)

    #STEP 3
    new_feature_engg_df = population_feature_engineering(new_county_df, fip)
    
    #STEP 4: Merge rolling data frame for non-cumulative features with repsective rows of cumuluative features
    # print(new_feature_engg_df[['actuals.cases','actuals.newCases']].head(57).tail(45))
    #Split into cumulative and non cumulative:
    cumulative_columns = ['actuals.cases','actuals.deaths', 'metrics.caseDensity', 'metrics.deathDensity', 
                          'metrics.vaccinationsCompletedRatio', 'metrics.vaccinationsInitiatedRatio']
    rolling_input_df = new_feature_engg_df.loc[:, ~new_feature_engg_df.columns.isin(cumulative_columns)]
    non_rolling_input_df = new_feature_engg_df[cumulative_columns]
    
    #STEP 5
    #Pass right data frame to rolling
    rolling_7d_df = seven_day_rolling_average(rolling_input_df)
    
    #STEP 6
    # clean_fips(rolling_7d_df)
    
    #STEP 7
    final_df = merge_cumulative_noncumulative(rolling_7d_df,non_rolling_input_df) #Merging both dataframes

    #STEP 8
    new_df_list.append(final_df) #Appending the final dataframe for each county
    
    
    # #STEP 8: INCONSITENCIES 
    # check_inconsistencies(final_df)

    i+=1
    if i==1:
        break

print(len(new_df_list))

  0%|          | 0/3222 [00:00<?, ?it/s]

1


In [416]:
new_df_list[0]

,fips,metrics.infectionRate,metrics.testPositivityRatio,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.deathDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
2020-03-21,41001.0,1.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-03-28,41001.0,1.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-04,41001.0,1.000000,0.056000,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-11,41001.0,1.000000,0.056000,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-18,41001.0,1.000000,0.056000,NaN,NaN,NaN,1.0,0.0,0.0,0.000000,0.007,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-11,41001.0,0.797143,0.069571,0.400000,0.400000,1.0,2240.0,35.0,20.4,0.002171,0.680,0.844
2021-12-18,41001.0,0.890000,0.074000,0.457143,0.457143,1.0,2264.0,36.0,21.0,0.002233,0.683,0.848
2021-12-25,41001.0,1.090000,0.084857,0.400000,0.400000,1.0,2289.0,36.0,23.5,0.002233,0.685,0.851
2022-01-01,41001.0,1.295714,0.143571,0.514286,0.514286,1.0,2310.0,36.0,21.7,0.002233,0.687,0.853


In [406]:
import joblib
joblib.dump(new_df_list, "final_counties_timeseries_list.pkl")

['final_counties_timeseries_list.pkl']

In [407]:
county_df_list = joblib.load("final_counties_timeseries_list.pkl")

In [411]:
county_df_list[0]

,fips,metrics.infectionRate,metrics.testPositivityRatio,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.deathDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
2020-03-21,41001,7.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-03-28,41001,7.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-04,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-11,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-18,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-11,41001,5.58,0.487,2.8,2.8,7.0,2240.0,35.0,20.4,0.002171,0.680,0.844
2021-12-18,41001,6.23,0.518,3.2,3.2,7.0,2264.0,36.0,21.0,0.002233,0.683,0.848
2021-12-25,41001,7.63,0.594,2.8,2.8,7.0,2289.0,36.0,23.5,0.002233,0.685,0.851
2022-01-01,41001,9.07,1.005,3.6,3.6,7.0,2310.0,36.0,21.7,0.002233,0.687,0.853


In [402]:
new_df_list[0]

,fips,metrics.infectionRate,metrics.testPositivityRatio,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.deathDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
2020-03-21,41001,7.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-03-28,41001,7.00,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-04,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-11,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
2020-04-18,41001,7.00,0.392,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.007,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-11,41001,5.58,0.487,2.8,2.8,7.0,2240.0,35.0,20.4,0.002171,0.680,0.844
2021-12-18,41001,6.23,0.518,3.2,3.2,7.0,2264.0,36.0,21.0,0.002233,0.683,0.848
2021-12-25,41001,7.63,0.594,2.8,2.8,7.0,2289.0,36.0,23.5,0.002233,0.685,0.851
2022-01-01,41001,9.07,1.005,3.6,3.6,7.0,2310.0,36.0,21.7,0.002233,0.687,0.853


In [330]:
# statestimeseries_7d_rolling = statestimeseries.rolling(7, center=True).mean()
# start, end = '2020-03', '2022-01'
# fig, ax = plt.subplots()
# ax.plot(statestimeseries.loc[start:end, 'actuals.cases'], linestyle='-', 
#         linewidth=0.5, label='Daily observed cases')
# ax.plot(statestimeseries_week.loc[start:end, 'actuals.cases'], linestyle='-', label = 'Weekly observed actual cases')
# ax.plot(statestimeseries_7d_rolling.loc[start:end, 'actuals.cases'], linestyle='-', label='7d Rolling Average')
# ax.set_ylabel('Actual cases')
# ax.legend()

In [397]:
temp__DF = pd.read_csv("./counties.finalfeatures.timeseries.csv")

temp__DF.drop(['actuals.newCases','actuals.newDeaths'],axis=1,inplace=True)
temp__DF

,date,fips,state,county,actuals.cases,actuals.deaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.testPositivityRatio
0,2020-04-09,2013,AK,Aleutians East Borough,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
1,2020-04-10,2013,AK,Aleutians East Borough,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
2,2020-04-11,2013,AK,Aleutians East Borough,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
3,2020-04-12,2013,AK,Aleutians East Borough,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
4,2020-04-13,2013,AK,Aleutians East Borough,NaN,NaN,0.0,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131347,2022-01-07,56045,WY,Weston County,1263.0,16.0,17.0,0.347,0.389,1.13,2,2,3,0.087
2131348,2022-01-08,56045,WY,Weston County,1263.0,16.0,18.9,0.349,0.390,1.15,2,2,3,NaN
2131349,2022-01-09,56045,WY,Weston County,1263.0,16.0,20.8,0.349,0.390,1.17,2,2,3,NaN
2131350,2022-01-10,56045,WY,Weston County,1268.0,16.0,22.7,0.349,0.390,1.19,2,2,3,NaN


In [363]:
# temp_df = original_csv[original_csv['fips'] == 41001]
# # temp_df.head(10)

# temp_df['date'] = pd.DatetimeIndex(temp_df['date'].values)
# temp_df.index = pd.DatetimeIndex(temp_df['date'].values)
# final_df = temp_df.asfreq('D')
# final_df['date'] = final_df.index


In [366]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 631 entries, 2020-03-21 to 2022-01-12
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   date                                631 non-null    datetime64[ns]
 1   fips                                631 non-null    int64         
 2   state                               631 non-null    object        
 3   county                              631 non-null    object        
 4   actuals.cases                       616 non-null    float64       
 5   actuals.newCases                    614 non-null    float64       
 6   actuals.deaths                      616 non-null    float64       
 7   actuals.newDeaths                   615 non-null    float64       
 8   metrics.caseDensity                 629 non-null    float64       
 9   metrics.vaccinationsCompletedRatio  361 non-null    float64       
 10  metrics

In [347]:
count_cleaned_df.head(10)

,fips,state,county,actuals.cases,actuals.newCases,actuals.deaths,actuals.newDeaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,metrics.testPositivityRatio,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,date
1464363,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-21
1464364,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-22
1464365,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-23
1464366,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-24
1464367,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-25
1464368,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-26
1464369,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-27
1464370,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-28
1464371,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-29
1464372,41001,OR,Baker County,1.0,0.0,0.0,0.0,0.0,0.007,0.03,1.0,0.0,0.0,0.0,0.0,2020-03-30


In [365]:
new_df_list[0].head(45)

,fips,actuals.cases,actuals.newCases,actuals.deaths,actuals.newDeaths,metrics.caseDensity,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio,metrics.infectionRate,metrics.testPositivityRatio,riskLevels.overall,riskLevels.caseDensity,cdcTransmissionLevel,metrics.deathDensity
date,,,,,,,,,,,,,,
2020-03-21,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.000,0.0,0.0,0.000000,0.000000
2020-03-28,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.000,0.0,0.0,0.000000,0.000000
2020-04-04,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.392,0.0,0.0,0.000000,0.000000
2020-04-11,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.392,0.0,0.0,0.000000,0.000000
2020-04-18,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.392,0.0,0.0,0.000000,0.000000
2020-04-25,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.392,0.0,0.0,0.000000,0.000000
2020-05-02,41001,7.0,0.0,0.0,0.0,0.0,0.049,0.210,7.00,0.398,0.0,0.0,1.000000,0.000000
2020-05-09,41001,9.0,2.0,0.0,0.0,1.8,0.049,0.210,7.00,0.208,0.2,0.2,1.000000,0.000000
2020-05-16,41001,7.0,0.0,0.0,0.0,12.6,0.049,0.210,7.00,0.000,1.2,1.2,2.000000,0.000000


In [267]:
# # new_cases_daily_total_modified = new_cases_daily_total.T.reset_index()
# new_df_list[0].set_index('date',inplace=True)
# new_df_list[0]
# new_cases_daily_total_modified = new_df_list[0].assign(Weeks = new_df_list[0]['date']).drop(columns = 'date')

# # new_cases_daily_total_modified['Weeks'] =  
# # new_cases_daily_total_modified['Weeks'].astype('datetime64[ns]')

# # new_cases_weekly_total_method_3 = new_cases_daily_total_modified.resample('W-Mon', label='left',closed = 'left', on='Weeks').sum()
# # display(new_cases_weekly_total_method_3.tail(5))

In [249]:
counties_info_df = pd.read_csv("./Data/counties.csv")
counties_pop_df = counties_info_df[["fips","population"]]
counties_pop_df.set_index("fips",inplace=True)
counties_pop_df

,population
fips,
2013,3337
2016,5634
2020,288000
2050,18386
2060,836
...,...
56037,42343
56039,23464
56041,20226


In [254]:
# first_county_dates = new_df_list[0].date.tolist()
# second_county_dates = new_df_list[2].date.tolist()
# def Diff(li1, li2):
#     return list(set(li1) - set(li2)) + list(set(li2) - set(li1))
# print(Diff(first_county_dates, second_county_dates))

In [ ]:
original_csv.to_csv("Processed.counties.data.csv", index=False)

In [255]:
# county_grouped = final_df_modified.groupby('county')['date', 'state'].apply(list).reset_index()